In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
# Load both left and right eye dataset
base_path = 'C:/Users/User/DeepLearning/DL-Project'
left_df = pd.read_excel(f'{base_path}/updated_left_eye.xlsx')
right_df = pd.read_excel(f'{base_path}/updated_right_eye.xlsx')


In [3]:
left_df.head()

,Unnamed: 0,Fundus,Diagnostic Keywords,N,D,G,C,A,H,M,O
0,0,0_left.jpg,cataract,0,0,0,1,0,0,0,0
1,1,1_left.jpg,normal fundus,1,0,0,0,0,0,0,0
2,2,2_left.jpg,laser spot，moderate non proliferative retinopathy,0,1,0,0,0,0,0,1
3,3,3_left.jpg,normal fundus,1,0,0,0,0,0,0,0
4,4,4_left.jpg,macular epiretinal membrane,0,0,0,0,0,0,0,1


In [4]:
right_df.head()

,Unnamed: 0,Fundus,Diagnostic Keywords,N,D,G,C,A,H,M,O
0,0,0_right.jpg,normal fundus,1,0,0,0,0,0,0,0
1,1,1_right.jpg,normal fundus,1,0,0,0,0,0,0,0
2,2,2_right.jpg,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0
3,3,3_right.jpg,branch retinal artery occlusion,0,0,0,0,0,0,0,1
4,4,4_right.jpg,mild nonproliferative retinopathy,0,1,0,0,0,0,0,0


In [5]:
df = pd.concat([left_df, right_df], axis=0)
df.head()


,Unnamed: 0,Fundus,Diagnostic Keywords,N,D,G,C,A,H,M,O
0,0,0_left.jpg,cataract,0,0,0,1,0,0,0,0
1,1,1_left.jpg,normal fundus,1,0,0,0,0,0,0,0
2,2,2_left.jpg,laser spot，moderate non proliferative retinopathy,0,1,0,0,0,0,0,1
3,3,3_left.jpg,normal fundus,1,0,0,0,0,0,0,0
4,4,4_left.jpg,macular epiretinal membrane,0,0,0,0,0,0,0,1


In [6]:
df.tail()

,Unnamed: 0,Fundus,Diagnostic Keywords,N,D,G,C,A,H,M,O
3478,3495,4686_right.jpg,proliferative diabetic retinopathy,0,1,0,0,0,0,0,0
3479,3496,4688_right.jpg,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0
3480,3497,4689_right.jpg,normal fundus,1,0,0,0,0,0,0,0
3481,3498,4690_right.jpg,mild nonproliferative retinopathy,0,1,0,0,0,0,0,0
3482,3499,4784_right.jpg,hypertensive retinopathy，age-related macular d...,0,0,0,0,1,1,0,0


In [7]:
df.shape


(6968, 11)

In [8]:
# Input pipeline
image_name = df['Fundus'].to_list()
image_name[:5]

['0_left.jpg', '1_left.jpg', '2_left.jpg', '3_left.jpg', '4_left.jpg']

In [9]:
labels = df.loc[:,['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']]
labels = labels.values

In [10]:
labels

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 1],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]], dtype=int64)

In [11]:
labels.shape

(6968, 8)

In [12]:
image_name = tf.constant(image_name)
labels = tf.constant(labels, dtype=tf.float32)

In [13]:
labels

<tf.Tensor: shape=(6968, 8), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)>

In [14]:
# Create a function to load the images using TensorFlow's `tf.io.read_file` function and `tf.image.decode_jpeg` function.
def load_image(image_path, label):
    base_path = 'E:/odir-dataset/ODIR-5K/ODIR-5K/cropped_training_images'
    image = tf.io.read_file(f'{base_path}/{image_path}')
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [1600, 1600])
    image /= 255.0  # normalize the pixels
    image = tf.expand_dims(image, axis=0) # Add a batch dimension to the image
#     label = tf.transpose(label)
    return image, label

In [15]:
# labels.T

In [16]:
# Load the images and their labels into a TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((image_name, labels))
dataset = dataset.map(load_image)

In [17]:
dataset

<MapDataset element_spec=(TensorSpec(shape=(1, 1600, 1600, 3), dtype=tf.float32, name=None), TensorSpec(shape=(8,), dtype=tf.float32, name=None))>

In [18]:
from keras.applications import VGG19
base_model = VGG19(
    include_top=False,
    weights='imagenet',
    input_shape=(1600,1600,3)
)
base_model.trainable = False

In [21]:
# Define the model architecture
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(8, activation='sigmoid')
])

In [22]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 50, 50, 512)       20024384  
                                                                 
 flatten_2 (Flatten)         (None, 1280000)           0         
                                                                 
 dense_4 (Dense)             (None, 1024)              1310721024
                                                                 
 dense_5 (Dense)             (None, 8)                 8200      
                                                                 
Total params: 1,330,753,608
Trainable params: 1,310,729,224
Non-trainable params: 20,024,384
_________________________________________________________________


In [24]:
# Train the model
history = model.fit(dataset, epochs=2)

Epoch 1/2


ValueError: in user code:

    File "C:\Users\User\anaconda3\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\User\anaconda3\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\User\anaconda3\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\User\anaconda3\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\User\anaconda3\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "C:\Users\User\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "C:\User